# Implementing MNIST digit recognition in tensorflow without using Keras API

In [8]:
import tensorflow as tf
%colors nocolor



## A Simple Dense Class 

Dense layer implements the following input transformation, where W and b are model parameters, and activation is an element-wise function (usually relu, but it would be softmax for the last layer):

       output = activation(dot(W, input) + b)

In [ ]:
# implementing simple Python class, NaiveDense, 
# that creates two TensorFlow variables, W and b, 
# and exposes a __call__() method that applies the preceding transformation

class NaiveDense:
    def __init__(self, input_size , output_size , activation ):
        self.activation = activation
        
        # Create a matrix, W, of shape (input_size, output_size), initialized with random values.
        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)
        
        # Create a vector, b, of shape (output_size,), initialized with zeros.
        b_shape = (output_size,)
        b_intial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_intial_value)
        
    # Applying the forward pass    
    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)
    
    @property
    # Convenience method for retrieving the layer’s weights
    def weights(self):
        return [self.W, self.b]
    

## A simple Sequential Class
 
 Create a NaiveSequential class to chain these layers. It wraps a list of layers and exposes a __call__() method that simply calls the underlying layers on the inputs, in order. It also features a weights property to easily keep track of the layers’ parameters.

In [ ]:
class NaiveSequential:
    def __init__(self,layers) :
        self.layers = layers
        
    def __call__(self,inputs):
        x = inputs 
        for layer in self.layers :
            x = layer(x)
        return x
    
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights 
        return weights 
    


In [ ]:
# Using this NaiveDense class and this NaiveSequential class, we can create a mock 
# Keras model:

model = NaiveSequential([
    NaiveDense(input_size=28*28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])

assert len(model.weights) == 4


## Batch Generator

A way to iterate over the MNIST data in mini-batches

In [ ]:
import math
class BatchGenerator:
    def __init__(self, images, labels, batch_size = 128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images)/batch_size)
        
    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels 
        